# E-Commerce Website Conversion Funnel Analysis

## Overview
This project analyses user behaviour on the Cool T-Shirts e-commerce website using four event logs: site visits, add to cart, checkout, and purchase. The aim is to map the conversion funnel, calculate drop-off rates at each stage, and identify opportunities to improve sales.

**Skills Demonstrated:**
- Loading and merging multiple timestamped datasets
- Handling duplicates and null values
- Calculating funnel conversion and drop-off rates
- Time-difference analysis
- Deriving actionable business insights

**Tools Used:** Python (Pandas), Power BI (Dashboard Visualisation)

In [1]:
# Importing necessary libraries
import pandas as pd
import os

## 1. Loading in the CSV data

In [2]:
# Assigning each dataset to a Dataframe
visits = pd.read_csv('visits.csv', parse_dates=['visit_time'])
cart = pd.read_csv('cart.csv', parse_dates=['cart_time'])
checkout = pd.read_csv('checkout.csv', parse_dates=['checkout_time'])
purchase = pd.read_csv('purchase.csv', parse_dates=['purchase_time'])

# Quick glance at the Data
print("Visits:", visits.shape)
print("Cart:", cart.shape)
print("Checkout:", checkout.shape)
print("Purchase:", purchase.shape)

Visits: (2000, 2)
Cart: (348, 2)
Checkout: (226, 2)
Purchase: (252, 2)


## 2. Merging the dataframes (left joins on user_id)

In [3]:
# Merging the Dataframes using chained Merges
all_data = visits.merge(cart, on='user_id', how='left')\
                 .merge(checkout, on='user_id', how='left')\
                 .merge(purchase, on='user_id', how='left')

# Glance at the new merged Dataframe
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT


## 3. Calculating funnel counts

In [4]:
total_visits = len(all_data)
in_cart = all_data['cart_time'].notnull().sum()
in_checkout = all_data['checkout_time'].notnull().sum()
purchased = all_data['purchase_time'].notnull().sum()

print(f"Total visits: {total_visits}")
print(f"Added to cart: {in_cart}")
print(f"Reached checkout: {in_checkout}")
print(f"Completed purchase: {purchased}")

Total visits: 2108
Added to cart: 456
Reached checkout: 334
Completed purchase: 252


## 4. Step conversion and drop-off rates

In [5]:
# Calculating conversion and drop-off rates as percentages of previous stages
visit_to_cart = in_cart / total_visits * 100
cart_to_checkout = in_checkout / in_cart * 100
checkout_to_purchase = purchased / in_checkout * 100
overall_conversion = purchased / total_visits * 100

# Printing the percentages rounded to 2 d.p
print(f"Visit → Cart: {visit_to_cart:.2f}%")
print(f"Cart → Checkout: {cart_to_checkout:.2f}%")
print(f"Checkout → Purchase: {checkout_to_purchase:.2f}%")
print(f"Overall Visit → Purchase: {overall_conversion:.2f}%")

Visit → Cart: 21.63%
Cart → Checkout: 73.25%
Checkout → Purchase: 75.45%
Overall Visit → Purchase: 11.95%


## 5. Average time from visit to purchase

In [9]:
# Calculate time difference for users who purchased
all_data['time_to_purchase'] = all_data['purchase_time'] - all_data['visit_time']
all_data['time_to_purchase_minutes'] = all_data['time_to_purchase'].dt.total_seconds() / 60

# Calculating mean time to purchase
average_time = all_data['time_to_purchase_minutes'].mean()

print("Average time to purchase:", average_time)

0     NaN
1    44.0
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
Name: time_to_purchase_minutes, dtype: float64
Average time to purchase: 43.20634920634921


## 6. Exporting processed data

In [11]:
# Creating processed folder in working directory with error check incase folder exists
os.makedirs('processed', exist_ok=True)

# Saving dataframe all_data to processed folder and removing row numbers 
all_data.to_csv('processed/funnel_all_data.csv', index=False)
print("Updated CSV saved with minutes column!")

Updated CSV saved with minutes column!
